# **Simple Guardrails Model**

In [1]:
# install this if not already installed

#!pip install  openai -U

To get a openai api_key: https://platform.openai.com/api-keys


In [2]:
import os
from openai import OpenAI

client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY"),
)


In [3]:
import numpy as np

# model for phosphorus (P) removal efficiency and effluent P concentration
def compute_removal_eff_and_effluent_P(influent_P, chemical_dose, temperature, pH, dissolved_O2, SRT):
    removal_eff = np.clip(
            0.45*(chemical_dose/20) +
            0.15*np.clip((temperature-10)/15,0,1) +
            0.15*(1-abs(pH-7)/2) +
            0.15*np.clip(dissolved_O2/2,0,1) +
            0.10*np.clip(SRT/20,0,1), 0, 0.95)
    effluent_P = influent_P * (1 - removal_eff)
    return effluent_P, removal_eff

In [4]:
# function specifications

tools = [
    {
      "type": "function",
      "function": {
          "name": "compute_removal_eff_and_effluent_P",
          "description": "Computes the removal efficiency and effluent Phosphorus concentration in a wastewater treatment process",
          "parameters": {
              "type": "object",
              "properties": {
                  # commented parameters are unused for now
                #   "influent_flow": {
                #       "type": "number",
                #       "description": "The influent flow rate in cubic meters per second (m³/s)",
                #   },
                  "influent_P": {
                      "type": "number",
                      "description": "The influent Phosphorus concentration in micrograms per liter (µg/L)",
                  },
                  "chemical_dose": {
                      "type": "number",
                      "description": "The chemical dose in milligrams per liter (mg/L)",
                  },
                  "temperature": {
                      "type": "number",
                      "description": "The temperature of the influent in degrees Celsius (°C)",
                  },
                  "pH": {
                      "type": "number",
                      "description": "The pH level of the influent, representing its acidity or alkalinity",
                  },
                  "dissolved_O2": {
                      "type": "number",
                      "description": "The concentration of dissolved oxygen, measured in milligrams per liter (mg/L)",
                  },
                  "SRT": {
                      "type": "number",
                      "description": "The solids retention time (SRT) in days (d)",
                  },
                #   "HRT": {
                #       "type": "number",
                #       "description": "The hydraulic retention time (HRT) in hours (h)",
                #  },
              },
              #"required": ["influent_flow", "influent_P", "chemical_dose", "temperature", "pH", "dissolved_O2", "SRT", "HRT"],
              "required": ["influent_P", "chemical_dose", "temperature", "pH", "dissolved_O2", "SRT"],
          },
      },
    },
]

In [5]:
with open("phosphorus_run/set01_paper.md", "r") as file:
    paper = file.read()

paper

"# Statistical Analysis of Wastewater Treatment Efficiency\n\n## Abstract\n\nThis study presents a comprehensive statistical analysis of data collected from 500 trials in a wastewater treatment process, focusing on the removal efficiency of phosphorus (P). Key parameters analyzed include\ninfluent flow rate, influent phosphorus concentration, chemical dose, temperature, pH, dissolved oxygen, solids retention time (SRT), hydraulic retention time (HRT), removal efficiency, and effluent phosphorus concentration. The mean influent flow\nwas observed at 4793.05 m3/d, with an influent phosphorus concentration of 8.51 mg/L. The chemical dosing averaged at 17.76 mg/L, and the process operated at an average temperature of 18.97°C,\nwith a pH of 6.99. The dissolved oxygen level was maintained at 2.01 mg/L on average. The system's SRT and HRT were 14.1 days and 7.96 hours, respectively. Notably, the\naverage phosphorus removal efficiency was 83%, resulting in an effluent phosphorus concentration 

In [6]:
# prompt to extract inputs and model parameters from the paper
prompt_x_theta = """Given the following paper, please extract the parameters needed to compute the removal efficiency 
        and effluent Phosphorus concentration in a wastewater treatment process. The parameters 
        are: influent P, chemical dose, temperature, pH, dissolved O2 and SRT. 
        The paper is as follows:\n\n""" + paper
openai_response_x_theta = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt_x_theta}],
    #add our function specs to the model
    tools=tools
)

response_x=openai_response_x_theta.choices[0].message
response_x

ChatCompletionMessage(content='I will extract the parameters needed to compute the removal efficiency and effluent Phosphorus concentration from the provided paper. The parameters are influent P, chemical dose, temperature, pH, dissolved O2, and SRT.\n\nFrom the paper, the extracted values are:\n- Influent Phosphorus Concentration: 8.51 mg/L\n- Chemical Dose: 17.76 mg/L\n- Temperature: 18.97°C\n- pH: 6.99\n- Dissolved Oxygen: 2.01 mg/L\n- Solids Retention Time (SRT): 14.1 days\n\nNow, I will use these extracted parameters to compute the removal efficiency and effluent Phosphorus concentration in a wastewater treatment process.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_tZruGj5yROnxWatXcv7HNpCe', function=Function(arguments='{"influent_P":8.51,"chemical_dose":17.76,"temperature":18.97,"pH":6.99,"dissolved_O2":2.01,"SRT":14.1}', name='compute_removal_eff_and_effluent_P'), type='function')])

In [7]:
openai_response_x_theta.choices[0].message.tool_calls[0].function

Function(arguments='{"influent_P":8.51,"chemical_dose":17.76,"temperature":18.97,"pH":6.99,"dissolved_O2":2.01,"SRT":14.1}', name='compute_removal_eff_and_effluent_P')

In [8]:
import json

fun_response_x_theta = json.loads(openai_response_x_theta.choices[0].message.tool_calls[0].function.arguments)
fun_response_x_theta

{'influent_P': 8.51,
 'chemical_dose': 17.76,
 'temperature': 18.97,
 'pH': 6.99,
 'dissolved_O2': 2.01,
 'SRT': 14.1}

In [9]:
influent_P = fun_response_x_theta['influent_P']
chemical_dose = fun_response_x_theta['chemical_dose']
temperature = fun_response_x_theta['temperature']
pH = fun_response_x_theta['pH']
dissolved_O2 = fun_response_x_theta['dissolved_O2']
SRT = fun_response_x_theta['SRT']

effluent_P_pred, removal_eff_pred = compute_removal_eff_and_effluent_P(influent_P, chemical_dose, temperature, pH, dissolved_O2, SRT)
effluent_P_pred, removal_eff_pred

(1.1994844999999992, 0.8590500000000001)

In [10]:
tools_y = [ {
      "type": "function",
      "function": {
          "name": "get_true_removal_eff_and_effluent_P",
          "description": "Extracts true values of removal efficiency and effluent Phosphorus concentration from the paper",
          "parameters": {
              "type": "object",
              "properties": {
                "removal_eff": {
                    "type": "number",
                    "description": "The removal efficiency of Phosphorus in the wastewater treatment process, expressed as a decimal fraction (0 to 1)",
                },
                "effluent_P": {
                    "type": "number",
                    "description": "The effluent Phosphorus concentration in milligrams per liter (mg/L)",
                },
              },
              "required": ["removal_eff", "effluent_P"],
          },
      },
    },
]

In [11]:
prompt_y = """Given the following paper, extract the removal 
        efficiency and effluent Phosphorus concentration in mg/L.""" + paper

openai_response_y = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt_y}],
    #add our function specs to the model
    tools=tools_y
)

response=openai_response_y.choices[0].message
response

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_y50yvI2epgLFi18K4CChlFdu', function=Function(arguments='{"removal_eff":0.83,"effluent_P":1.44}', name='get_true_removal_eff_and_effluent_P'), type='function')])

In [12]:
openai_response_y.choices[0].message.tool_calls[0].function

Function(arguments='{"removal_eff":0.83,"effluent_P":1.44}', name='get_true_removal_eff_and_effluent_P')

In [13]:
fun_response=json.loads(openai_response_y.choices[0].message.tool_calls[0].function.arguments)
fun_response

{'removal_eff': 0.83, 'effluent_P': 1.44}

In [14]:
removal_eff_true = fun_response['removal_eff']
effluent_P_true = fun_response['effluent_P']
removal_eff_true, effluent_P_true

(0.83, 1.44)

In [15]:
def compute_error_effluent_P(effluent_P_true, effluent_P_pred):
    """
    Computes the error in effluent Phosphorus concentration.
    
    Parameters:
    - effluent_P_true: The true effluent Phosphorus concentration in mg/L.
    - effluent_P_pred: The effluent Phosphorus concentration in mg/L predicted by simulation.
    
    Returns:
    - error: The absolute error in effluent Phosphorus concentration.
    """
    return abs(effluent_P_true - effluent_P_pred)

compute_error_effluent_P(effluent_P_true, effluent_P_pred)

0.24051550000000077